In [179]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )


python version: 2.7.13 |Anaconda 4.3.0 (32-bit)| (default, Dec 20 2016, 23:08:16) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
pandas version: 0.19.2
magellan version: 0.1.0


In [180]:
#Get the paths
path_google = "/home/gaurav/DS/stage3/csv_data/" + "playstore_formatted.csv"
path_apple = "/home/gaurav/DS/stage3/csv_data/" + "appstore_formatted.csv"

print path_google
print path_apple

/home/gaurav/DS/stage3/csv_data/playstore_formatted.csv
/home/gaurav/DS/stage3/csv_data/appstore_formatted.csv


In [181]:
# Load csv files as dataframes and set the key attribute in the dataframe
google = em.read_csv_metadata(path_google, key='ID')
apple = em.read_csv_metadata(path_apple, key='ID')

print('Number of tuples in table "google:" ' + str(len(google)))
print('Number of tuples in table "apple:" ' + str(len(apple)))
print('Number of tuples in "google" X "apple" (i.e the cartesian product): ' + str(len(google)*len(apple)))

Number of tuples in table "google:" 3789
Number of tuples in table "apple:" 3697
Number of tuples in "google" X "apple" (i.e the cartesian product): 14007933


In [182]:
#Printing few rows
google.head(2)
apple.head(2)

,ID,name,category,developer,rating
0,b1,Candy Crush Saga,Entertainment,King,4.70807
1,b2,Super Mario Run,Action,"Nintendo Co., Ltd.",2.97468


In [183]:
# Display the keys of the input tables
em.get_key(google), em.get_key(apple)

('ID', 'ID')

In [184]:
# Blocking plan

# A, B -- Overlap blocker [name] --------------------|---> candidate set

In [185]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
cand_1 = ob.block_tables(google, apple, 'name', 'name', 
                    l_output_attrs=['name', 'category', 'developer', 'rating'], 
                    r_output_attrs=['name', 'category', 'developer', 'rating'],
                    overlap_size=2, show_progress=False
                    )

# Again applying blocking using 'developer' attribute
cand_2 = ob.block_candset(cand_1, 'developer', 'developer', word_level=True, overlap_size=1, show_progress=False)

len(cand_2)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


4307

In [186]:
#Converting to .csv file
cand_2.to_csv('cand_2.csv')

In [188]:
#Reading back the file after adding the labels
read_back = em.read_csv_metadata('cand_2.csv', 
                         key='_id',
                         ltable=google, rtable=apple, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

len(read_back)

4307

In [189]:
# Split the data in read_back into development set (I) and evaluation set (J)
IJ = em.split_train_test(read_back, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [190]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

In [191]:
# Generate features
feature_table = em.get_features_for_matching(google, apple)

In [192]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=False)

In [193]:
# Display first few rows
H.head(3)

,_id,ltable_ID,rtable_ID,ID_ID_lev_dist,ID_ID_lev_sim,ID_ID_jar,ID_ID_jwn,ID_ID_exm,ID_ID_jac_qgm_3_qgm_3,name_name_jac_qgm_3_qgm_3,...,category_category_sw,developer_developer_jac_qgm_3_qgm_3,developer_developer_cos_dlm_dc0_dlm_dc0,developer_developer_jac_dlm_dc0_dlm_dc0,developer_developer_mel,developer_developer_lev_dist,developer_developer_lev_sim,developer_developer_nmw,developer_developer_sw,label
3018,30692,a2306,b2285,4,0.2,0.466667,0.466667,0,0.0,0.243590,...,2.0,0.063492,0.192450,0.090909,0.504762,44.0,0.120000,-30.0,6.0,0
1364,11078,a1501,b1178,4,0.2,0.466667,0.466667,0,0.0,0.100000,...,1.0,0.137931,0.353553,0.200000,0.473430,19.0,0.173913,-13.0,4.0,0
3817,37880,a734,b3118,5,0.0,0.483333,0.483333,0,0.0,0.025641,...,1.0,0.160000,0.353553,0.200000,0.312865,15.0,0.210526,-9.0,4.0,0


In [194]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0xa7b2556c>,5,0.979592,0.972603,0.967213,0.945205,1.000000,0.972923
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0xa7c4cdcc>,5,1.000000,1.000000,1.000000,0.985294,1.000000,0.997059
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0xa7c4cd6c>,5,0.895833,1.000000,0.962264,0.981132,0.918367,0.951519
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0xa7c4c74c>,5,0.903846,0.985294,1.000000,0.985075,0.932203,0.961284
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0xa7c4cfcc>,5,0.886792,0.957746,0.904762,0.984848,0.948276,0.936485
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0xa7c4ca0c>,5,0.405172,0.511278,0.468750,0.485294,0.528846,0.479868


In [195]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0xa7b2556c>,5,1.000000,0.986111,0.936508,0.985714,0.983607,0.978388
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0xa7c4cdcc>,5,1.000000,0.972222,0.920635,0.957143,0.967213,0.963443
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0xa7c4cd6c>,5,0.895833,0.861111,0.809524,0.742857,0.737705,0.809406
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0xa7c4c74c>,5,0.979167,0.930556,0.920635,0.942857,0.901639,0.934971
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0xa7c4cfcc>,5,0.979167,0.944444,0.904762,0.928571,0.901639,0.931717
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0xa7c4ca0c>,5,0.979167,0.944444,0.952381,0.942857,0.901639,0.944098


In [196]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='label', show_progress=False)

In [197]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

In [198]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

In [199]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 99.43% (175/176)
Recall : 98.31% (175/178)
F1 : 98.87%
False positives : 1 (out of 176 positive predictions)
False negatives : 3 (out of 1117 negative predictions)
